# 1. Import thư viện

In [1]:
import cv2
import numpy as np
import os
from PIL import Image
import shutil
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torch.nn.parallel import DataParallel
from torch.cuda.amp import autocast, GradScaler
import torch.nn.functional as F
from torchvision.utils import save_image
import torchsummary
from tqdm import tqdm
from models.srcnn import *
from models.idn import *

from models.vdsr import *

from models.sr_model import *
from models.fsrcnn import *
from models.wdsr import *
from models.e2dsr import *

import time



In [2]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 2. Tạo Mô hình SR

In [3]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
# device = torch.device('cpu')

In [4]:
e2dsr_sobel = E2DSR(edge_option='sobel').to(device)
e2dsr_canny = E2DSR(edge_option='canny').to(device)

In [5]:

# e2dsr_canny.load_state_dict(torch.load('weight/best_e2dsr_canny.pth', map_location=device))
# e2dsr_canny.load_state_dict(torch.load('weight/best_e2dsr_cannyx4_model.pth', map_location=device))
# edrn_sobel.load_state_dict(torch.load('weight/best_sobel_srx4_model.pth', map_location=device))
# edrn_canny.load_state_dict(torch.load('weight/best_canny_srx4_model.pth', map_location=device))
# e2dsr_sobel.load_state_dict(torch.load('best_e2dsr_sobelbel.pth', map_location=device))
# vdsr.load_state_dict(torch.load('weight/best_vdsr.pth', map_location=device))

In [6]:
class ImageDataset(Dataset):
    def __init__(self, lr_dir, hr_dir):
        self.lr_files = sorted(os.listdir(lr_dir))
        self.hr_files = sorted(os.listdir(hr_dir))
        self.lr_dir = lr_dir
        self.hr_dir = hr_dir

    def __len__(self):
        return len(self.lr_files)

    def __getitem__(self, idx):
        lr_image = Image.open(os.path.join(self.lr_dir, self.lr_files[idx])).convert('RGB')
        hr_image = Image.open(os.path.join(self.hr_dir, self.hr_files[idx])).convert('RGB')
        
        # size = 32
        # transform_hr = transforms.Compose([
        #     transforms.ToPILImage(),
        #     transforms.CenterCrop((size*4, size*4)),
        #     transforms.ToTensor()
        # ])

        # transform_lr = transforms.Compose([
        #     transforms.ToPILImage(),
        #     transforms.CenterCrop((size, size)),
        #     # transforms.Resize((size*4, size*4)),
        #     transforms.ToTensor()
        # ])
        
        # lr_image = transform_lr(lr_image)
        # hr_image = transform_hr(hr_image)
        transform = transforms.Compose([
            # transforms.ToPILImage(),
            transforms.ToTensor()
        ])
        
        lr_image = transform(lr_image)
        hr_image = transform(hr_image)
        return lr_image, hr_image

# 3. Tạo Hyperparameter

In [7]:
# Đường dẫn tới bộ dữ liệu
train_lr_dir = 'Train/LR'
train_hr_dir = 'Train/HR'
valid_lr_dir = 'Test/LR'
valid_hr_dir = 'Test/HR'
# test_hr_dir  = '/kaggle/input/srdataset/sr_data/test/HR'
# test_lr_dir  = '/kaggle/input/srdataset/sr_data/test/LR'

# print(torch.cuda.memory_allocated())
# print(torch.cuda.memory_reserved())

In [8]:
from torch.amp import autocast, GradScaler
scaler = GradScaler()

# Khởi tạo dataset và dataloader
train_dataset = ImageDataset(train_lr_dir, train_hr_dir)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

valid_dataset = ImageDataset(valid_lr_dir, valid_hr_dir)
valid_loader = DataLoader(valid_dataset)

print(len(train_loader))

# Khởi tạo loss function
criterion = nn.MSELoss()

# Khởi tạo optimizers, schedulers cho từng mô hình
optim_e2dsr_canny = optim.Adam(e2dsr_canny.parameters(), lr=1e-4, betas=(0.9, 0.999))
scheduler_e2dsr_canny = optim.lr_scheduler.StepLR(optim_e2dsr_canny, step_size=10**5, gamma=0.5)

optim_e2dsr_sobel = optim.Adam(e2dsr_sobel.parameters(), lr=1e-4, betas=(0.9, 0.999))
scheduler_e2dsr_sobel = optim.lr_scheduler.StepLR(optim_e2dsr_sobel, step_size=10**5, gamma=0.5)

# optim_vdsr = optim.Adam(vdsr.parameters(), lr=1e-4, betas=(0.9, 0.999))
# scheduler_vdsr = optim.lr_scheduler.StepLR(optim_vdsr, step_size=10**5, gamma=0.5)

# optim_fsrcnn = optim.Adam(fsrcnn.parameters(), lr=1e-4, betas=(0.9, 0.999))
# scheduler_fsrcnn = optim.lr_scheduler.StepLR(optim_fsrcnn, step_size=10**5, gamma=0.5)

12500


In [9]:
def calculate_psnr(img1, img2):
    mse = torch.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    max_pixel = 1.0
    psnr = 20 * torch.log10(max_pixel / torch.sqrt(mse))
    return psnr.item()

In [10]:
print(device)

cuda


# 4. Training

In [11]:


# Hàm tính PSNR
def calculate_psnr(img1, img2):
    mse = torch.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    max_pixel = 1.0
    psnr = 20 * torch.log10(max_pixel / torch.sqrt(mse))
    return psnr.item()

num_epochs = 24

best_psnr_e2dsr_canny = float('-inf')
best_psnr_e2dsr_sobel = float('-inf')
best_psnr_vdsr = float('-inf')
best_psnr_fsrcnn = float('-inf')
torch.cuda.empty_cache()

losses_e2dsr_canny = []
losses_e2dsr_sobel = []
losses_vdsr = []
losses_fsrcnn = []

avg_psnr_e2dsr_canny = []
avg_psnr_e2dsr_sobel = []
avg_psnr_vdsr = []
avg_psnr_fsrcnn = []

val_avg_psnr_e2dsr_canny = []
val_avg_psnr_e2dsr_sobel = []
val_avg_psnr_vdsr = []
val_avg_psnr_fsrcnn = []

patience = 50
epochs_no_improve = 0
log_file = open('training_log_models.txt', 'a')

for epoch in range(num_epochs):
    e2dsr_canny.train()
    e2dsr_sobel.train()
    # vdsr.train()
    # fsrcnn.train()

    epoch_loss_e2dsr_canny, psnr_values_e2dsr_canny = 0, 0
    epoch_loss_e2dsr_sobel, psnr_values_e2dsr_sobel = 0, 0
    epoch_loss_vdsr, psnr_values_vdsr = 0, 0
    epoch_loss_fsrcnn, psnr_values_fsrcnn = 0, 0
    start_time = time.time()
    torch.cuda.empty_cache()
    # Training loop for each model
    for i, (lr_images, hr_images) in enumerate(tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch')):
        lr_images = lr_images.to(device)
        hr_images = hr_images.to(device)

        # # Train e2dsr_canny model
        optim_e2dsr_canny.zero_grad()
        with autocast(device_type='cuda'):
            outputs_e2dsr_canny = e2dsr_canny(lr_images)
            loss_e2dsr_canny = criterion(outputs_e2dsr_canny, hr_images)
        psnr_e2dsr_canny = calculate_psnr(outputs_e2dsr_canny, hr_images)

        scaler.scale(loss_e2dsr_canny).backward()
        scaler.step(optim_e2dsr_canny)
        scaler.update()
        scheduler_e2dsr_canny.step()

        epoch_loss_e2dsr_canny += loss_e2dsr_canny.item()
        
        psnr_values_e2dsr_canny += psnr_e2dsr_canny
        # while i < 12500/2:
        # Train e2dsr_sobel model
        optim_e2dsr_sobel.zero_grad()
        with autocast(device_type='cuda'):
            outputs_e2dsr_sobel = e2dsr_sobel(lr_images)
            loss_e2dsr_sobel = criterion(outputs_e2dsr_sobel, hr_images)
        psnr_e2dsr_sobel = calculate_psnr(outputs_e2dsr_sobel, hr_images)

        scaler.scale(loss_e2dsr_sobel).backward()
        scaler.step(optim_e2dsr_sobel)
        scaler.update()
        scheduler_e2dsr_sobel.step()

        epoch_loss_e2dsr_sobel += loss_e2dsr_sobel.item()
        psnr_values_e2dsr_sobel += psnr_e2dsr_sobel

        # Train RDN model
        # optim_vdsr.zero_grad()
        # with autocast(device_type='cuda'):
        #     outputs_vdsr = vdsr(lr_images)
        #     loss_vdsr = criterion(outputs_vdsr, hr_images)
        # psnr_vdsr = calculate_psnr(outputs_vdsr, hr_images)

        # scaler.scale(loss_vdsr).backward()
        # scaler.step(optim_vdsr)
        # scaler.update()
        # scheduler_vdsr.step()

        # epoch_loss_vdsr += loss_vdsr.item()
        # psnr_values_vdsr += psnr_vdsr

        # # Train FSRCNN model
        # optim_fsrcnn.zero_grad()
        # with autocast(device_type='cuda'):
        #     outputs_fsrcnn = fsrcnn(lr_images)
        #     loss_fsrcnn = criterion(outputs_fsrcnn, hr_images)
        # psnr_fsrcnn = calculate_psnr(outputs_fsrcnn, hr_images)

        # scaler.scale(loss_fsrcnn).backward()
        # scaler.step(optim_fsrcnn)
        # scaler.update()
        # scheduler_fsrcnn.step()

        # epoch_loss_fsrcnn += loss_fsrcnn.item()
        # psnr_values_fsrcnn += psnr_fsrcnn

    # Average losses and PSNRs
    avg_epoch_loss_e2dsr_canny = epoch_loss_e2dsr_canny / len(train_loader)
    avg_psnr_e2dsr_canny_epoch = psnr_values_e2dsr_canny / len(train_loader)
    losses_e2dsr_canny.append(avg_epoch_loss_e2dsr_canny)
    avg_psnr_e2dsr_canny.append(avg_psnr_e2dsr_canny_epoch)

    avg_epoch_loss_e2dsr_sobel = epoch_loss_e2dsr_sobel / len(train_loader)
    avg_psnr_e2dsr_sobel_epoch = psnr_values_e2dsr_sobel / len(train_loader)
    losses_e2dsr_sobel.append(avg_epoch_loss_e2dsr_sobel)
    avg_psnr_e2dsr_sobel.append(avg_psnr_e2dsr_sobel_epoch)

    # avg_epoch_loss_vdsr = epoch_loss_vdsr / len(train_loader)
    # avg_psnr_vdsr_epoch = psnr_values_vdsr / len(train_loader)
    # losses_vdsr.append(avg_epoch_loss_vdsr)
    # avg_psnr_vdsr.append(avg_psnr_vdsr_epoch)

    # avg_epoch_loss_fsrcnn = epoch_loss_fsrcnn / len(train_loader)
    # avg_psnr_fsrcnn_epoch = psnr_values_fsrcnn / len(train_loader)
    # losses_fsrcnn.append(avg_epoch_loss_fsrcnn)
    # avg_psnr_fsrcnn.append(avg_psnr_fsrcnn_epoch)

    # Validation for all models
    e2dsr_canny.eval()
    e2dsr_sobel.eval()
    # vdsr.eval()
    # fsrcnn.eval()

    val_psnr_e2dsr_canny, val_psnr_e2dsr_sobel = 0, 0
    val_psnr_vdsr, val_psnr_fsrcnn = 0, 0

    with torch.no_grad():
        for (lr_images, hr_images) in valid_loader:
            lr_images = lr_images.cuda()
            hr_images = hr_images.cuda()

            # # Validate e2dsr_canny
            outputs_e2dsr_canny = e2dsr_canny(lr_images)
            psnr_e2dsr_canny = calculate_psnr(outputs_e2dsr_canny, hr_images)
            val_psnr_e2dsr_canny += psnr_e2dsr_canny

            # Validate e2dsr_sobelbel
            outputs_e2dsr_sobel = e2dsr_sobel(lr_images)
            psnr_e2dsr_sobel = calculate_psnr(outputs_e2dsr_sobel, hr_images)
            val_psnr_e2dsr_sobel += psnr_e2dsr_sobel

            # # Validate RDN
            # outputs_vdsr = vdsr(lr_images)
            # psnr_vdsr = calculate_psnr(outputs_vdsr, hr_images)
            # val_psnr_vdsr += psnr_vdsr

            # # Validate FSRCNN
            # outputs_fsrcnn = fsrcnn(lr_images)
            # psnr_fsrcnn = calculate_psnr(outputs_fsrcnn, hr_images)
            # val_psnr_fsrcnn += psnr_fsrcnn

    val_avg_psnr_e2dsr_canny_epoch = val_psnr_e2dsr_canny / len(valid_loader)
    val_avg_psnr_e2dsr_canny.append(val_avg_psnr_e2dsr_canny_epoch)

    val_avg_psnr_e2dsr_sobel_epoch = val_psnr_e2dsr_sobel / len(valid_loader)
    val_avg_psnr_e2dsr_sobel.append(val_avg_psnr_e2dsr_sobel_epoch)

    # val_avg_psnr_rdn_epoch = val_psnr_rdn / len(valid_loader)
    # val_avg_psnr_rdn.append(val_avg_psnr_rdn_epoch)

    # val_avg_psnr_fsrcnn_epoch = val_psnr_fsrcnn / len(valid_loader)
    # val_avg_psnr_fsrcnn.append(val_avg_psnr_fsrcnn_epoch)

    # Save best model
    if val_avg_psnr_e2dsr_canny_epoch > best_psnr_e2dsr_canny:
        best_psnr_e2dsr_canny = val_avg_psnr_e2dsr_canny_epoch
        torch.save(e2dsr_canny.state_dict(), 'best_e2dsr_canny.pth')
        print(f"Saved e2dsr_cannyR model with PSNR {best_psnr_e2dsr_canny:.4f}")
    if val_avg_psnr_e2dsr_sobel_epoch > best_psnr_e2dsr_sobel:
        best_psnr_e2dsr_sobel = val_avg_psnr_e2dsr_sobel_epoch
        torch.save(e2dsr_sobel.state_dict(), 'best_e2dsr_sobel.pth')
        print(f"Saved e2dsr_sobel model with PSNR {best_psnr_e2dsr_sobel:.4f}")

    torch.save(e2dsr_canny.state_dict(), f'path/e2dsr_canny_{epoch}.pth')
    torch.save(e2dsr_sobel.state_dict(), f'path/e2dsr_sobel_{epoch}.pth')
    # if val_avg_psnr_rdn_epoch > best_psnr_rdn:
    #     best_psnr_rdn = val_avg_psnr_rdn_epoch
    #     torch.save(rdn.state_dict(), 'best_rdn.pth')
    #     print(f"Saved RDN model with PSNR {best_psnr_rdn:.4f}")
    # if val_avg_psnr_fsrcnn_epoch > best_psnr_fsrcnn:
    #     best_psnr_fsrcnn = val_avg_psnr_fsrcnn_epoch
    #     torch.save(fsrcnn.state_dict(), 'best_fsrcnn.pth')

    print(f"Epoch [{epoch+1}/{num_epochs}] completed: e2dsr_canny Loss: {avg_epoch_loss_e2dsr_canny:.4f}, PSNR: {avg_psnr_e2dsr_canny_epoch:.4f}, Validation PSNR: {val_avg_psnr_e2dsr_canny_epoch:.4f}, e2dsr_sobel Loss: {avg_epoch_loss_e2dsr_sobel:.4f}, PSNR: {avg_psnr_e2dsr_sobel_epoch:.4f}, Validation PSNR: {val_avg_psnr_e2dsr_sobel_epoch:.4f}")
# 
    log_file.write(f"Epoch {epoch+1}:  e2dsr_canny PSNR: {avg_psnr_e2dsr_canny_epoch:.4f}, Validation PSNR: {val_avg_psnr_e2dsr_canny_epoch:.4f}\n")
    log_file.write(f"              e2dsr_sobel PSNR: {avg_psnr_e2dsr_sobel_epoch:.4f}, Validation PSNR: {val_avg_psnr_e2dsr_sobel_epoch:.4f}\n")
    # log_file.write(f"Epoch {epoch+1}: RDN PSNR: {avg_psnr_rdn_epoch:.4f}, Validation PSNR: {val_avg_psnr_rdn_epoch:.4f}\n")
    # log_file.write(f"Epoch {epoch+1}: FSRCNN PSNR: {avg_psnr_fsrcnn_epoch:.4f}, Validation PSNR: {val_avg_psnr_fsrcnn_epoch:.4f}\n")

    log_file.flush()

log_file.close()


Epoch 1/24:   0%|          | 0/12500 [00:00<?, ?batch/s]

Epoch 1/24: 100%|██████████| 12500/12500 [32:06<00:00,  6.49batch/s]


Saved e2dsr_cannyR model with PSNR 26.8483
Saved e2dsr_sobel model with PSNR 26.9011
Epoch [1/24] completed: e2dsr_canny Loss: 0.0034, PSNR: 25.3268, Validation PSNR: 26.8483, e2dsr_sobel Loss: 0.0034, PSNR: 25.3597, Validation PSNR: 26.9011


Epoch 2/24: 100%|██████████| 12500/12500 [32:21<00:00,  6.44batch/s]


Saved e2dsr_cannyR model with PSNR 27.2017
Saved e2dsr_sobel model with PSNR 27.1822
Epoch [2/24] completed: e2dsr_canny Loss: 0.0026, PSNR: 26.0623, Validation PSNR: 27.2017, e2dsr_sobel Loss: 0.0026, PSNR: 26.0733, Validation PSNR: 27.1822


...

Epoch 23/24: 100%|██████████| 12500/12500 [32:10<00:00,  6.47batch/s]


Saved e2dsr_sobel model with PSNR 27.7584
Epoch [23/24] completed: e2dsr_canny Loss: 0.0023, PSNR: 26.7181, Validation PSNR: 27.7710, e2dsr_sobel Loss: 0.0023, PSNR: 26.7200, Validation PSNR: 27.7584


Epoch 24/24: 100%|██████████| 12500/12500 [32:11<00:00,  6.47batch/s]


Epoch [24/24] completed: e2dsr_canny Loss: 0.0023, PSNR: 26.7254, Validation PSNR: 27.7791, e2dsr_sobel Loss: 0.0023, PSNR: 26.7266, Validation PSNR: 27.7565


: 

# 5. Testing